# Usage of the solver

In [6]:
# General setup
from numpy.random import randint, choice

explain some intro stuff here maybe

In [7]:
# idk if random generation is the right idea,
# maybe a more systematic approach would be better.
# just trying things out for now.
class Model():
    def __init__(self, worlds=None, agents=None,
                 relations=None, valuations=None):
        self.worlds = worlds
        self.agents = agents
        self.relations = relations
        self.valuations = valuations
        if self.valuations is None:
            self.create_valuations()
        if self.relations is None:
            self.create_relations()
        if self.agents is None:
            self.create_agents()
        if self.worlds is None:
            self.create_worlds()

    def create_worlds(self, max: int = 12):
        # generate a random number of worlds
        self.worlds = []
        for i in range(randint(1, max + 1)):
            self.worlds.append(f"w{i}")
        return self.worlds

    def create_agents(self, max: int = 12):
        # generate a random number of agents
        self.agents = []
        for i in range(randint(1, max + 1)):
            self.agents.append(chr(ord('A')+i))
        return self.agents

    def create_relations(self):
        # generate random relations
        if self.worlds is None:
            self.create_worlds()
        if self.agents is None:
            self.create_agents()
        # This should make S5 models, doesn't do that yet
        self.relations = {}
        for a in self.agents:
            self.relations[a] = {}
            for world in self.worlds:
                self.relations[a][world] = {w for w in self.worlds
                                            if w == world
                                            or randint(0, 10) < 5}
        return self.relations

    def create_valuations(self, max: int = 3):
        # generate random valuations
        if self.worlds is None:
            self.create_worlds()

        atoms = [chr(ord('p')+i) for i in range(randint(1, max + 1))]
        self.valuations = {}
        for atom in atoms:
            self.valuations[atom] = {w for w in self.worlds
                                     if randint(0, 10) < 5}
        return self.valuations

In [8]:
def create_formula(model: Model) -> str:
    formula = ""

    for _ in range(randint(1,12)):
        formula += "K" + choice(model.agents)
    formula += choice(list(model.valuations.keys()))

    return formula

In [9]:
class Solver:
    def __init__(self, formula, model: Model = None, verbose: bool = False):
        """
        Initialize the solver with a starting formula, set of worlds, relations and valuations

        To Do: Add support for NOT operator, and, or etc.

        :param formula: Description
        :param model:
        :param verbose:
        """
        self.start = formula
        self.level = 0
        self.f = formula
        self.w = model.worlds if model.worlds else model.create_worlds()
        self.r = model.relations if model.relations else model.create_relations()
        self.v = model.valuations if model.valuations else model.create_valuations()
        self.talks = verbose

    def eval_formula(self, formula, world):
        if formula[0] == "p":
            return world in self.v.get("p", set())
        if formula[0] == "K":
            agent, sub_formula = formula[1], formula[2:]
            return all(self.eval_formula(sub_formula, connected_world)
                       for connected_world in self.r[agent][world])

    def satisfying_worlds(self):
        return {w for w in self.w if self.eval_formula(self.start, w)}

    def strip_outer_K(self):
        self.f = self.f[2:] if self.f[0] == "K" else self.f

    def analyze(self):
        answers = []

        while True:
            ws = self.satisfying_worlds()

            # print(f"\nToM level {self.level}")
            if self.talks:
                print("\n")
                print("Formula:", self.f)
                print("Possible worlds:", ws)

            if not ws:
                if self.talks:
                    print("→ inconsistent announcement")
                answers.append(None)
            elif len(ws) == 1:
                ans = next(iter(ws))
                if self.talks:
                    print("→ unique world:", ans)
                answers.append(ans)
            else:
                p_vals = {self.eval_formula(("p",), w) for w in ws}
                if self.talks:
                    if len(p_vals) == 1:
                        print("→ p is known =", p_vals.pop())
                    else:
                        print("→ no unique outcome")
                answers.append(None)

            if self.f[0] != "K":
                break

            self.strip_outer_K()
            self.level += 1

        unique_answers = [a for a in answers if a is not None]
        if (len(unique_answers) == len(set(unique_answers))
                and len(unique_answers) == len(answers)):
            print("\nUnique different answer found for each K operator removed.")
            print("Formula:", self.start)
            print("worlds:", self.w)
            print("relations:", self.r)
            print("valuations:", self.v)
            return True
        else:
            # print("\t\tNo unique different answer found for each K operator removed.")
            print('.', end='')
            return False


To Do: 
- make sure model x formula combinations aren't repeated?
- add support for not in the formula generator and solver

In [10]:
for i in range(100):
    model = Model()
    formula = create_formula(model)
    if Solver(formula, model).analyze():
        print(model, formula)


....................................................................................................